In [1]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install ttkbootstrap pillow matplotlib

Note: you may need to restart the kernel to use updated packages.


In [1]:
import tkinter as tk
from tkinter import ttk
import requests
from ttkbootstrap import Style
from PIL import Image, ImageTk
import io
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

def get_weather():
    city = city_entry.get()
    if city:
        
        api_key = "ca17ee95f2c0049e488aac3739df2eac"
        base_url = "http://api.openweathermap.org/data/2.5/weather?"
        complete_url = base_url + "q=" + city + "&appid=" + api_key + "&units=metric"
        response = requests.get(complete_url)
        data = response.json()
        
        if response.status_code == 200:
            temp = data['main']['temp']
            humidity = data['main']['humidity']
            weather_desc = data['weather'][0]['description']
            icon_code = data['weather'][0]['icon']
            
            
            icon_url = f"http://openweathermap.org/img/wn/{icon_code}@2x.png"
            icon_response = requests.get(icon_url)
            icon_image = Image.open(io.BytesIO(icon_response.content))
            icon_photo = ImageTk.PhotoImage(icon_image)
            
            weather_icon.config(image=icon_photo)
            weather_icon.image = icon_photo
            
            temp_label.config(text=f"{temp}°C")
            humidity_label.config(text=f"Humidity: {humidity}%")
            desc_label.config(text=weather_desc.capitalize())
            
            
            forecast_url = f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={api_key}&units=metric"
            forecast_response = requests.get(forecast_url)
            forecast_data = forecast_response.json()
            
            if forecast_response.status_code == 200:
                plot_temperature(forecast_data)
        else:
            result_label.config(text="City not found.")
    else:
        result_label.config(text="Please enter a city name.")

def plot_temperature(forecast_data):
    times = [item['dt_txt'] for item in forecast_data['list']]
    temps = [item['main']['temp'] for item in forecast_data['list']]
    
    fig, ax = plt.subplots(figsize=(6, 3))
    ax.plot(times, temps)
    ax.set_xlabel("Time")
    ax.set_ylabel("Temperature (°C)")
    ax.set_title("Temperature Forecast")
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    
    
    for widget in plot_frame.winfo_children():
        widget.destroy()
    
  
    canvas = FigureCanvasTkAgg(fig, master=plot_frame)
    canvas.draw()
    canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)

# GUI setup
root = tk.Tk()
root.title("Weather App")
root.geometry("1000x500")

style = Style(theme="flatly")

main_frame = ttk.Frame(root, padding="20")
main_frame.pack(fill=tk.BOTH, expand=True)

# City input field
city_frame = ttk.Frame(main_frame)
city_frame.pack(fill=tk.X, pady=(0, 20))

city_label = ttk.Label(city_frame, text="Enter city:", font=("Helvetica", 12))
city_label.pack(side=tk.LEFT)

city_entry = ttk.Entry(city_frame, font=("Helvetica", 12), width=30)
city_entry.pack(side=tk.LEFT, padx=(10, 0))

# Button to fetch weather data
submit_button = ttk.Button(city_frame, text="Get Weather", command=get_weather, style="primary.TButton")
submit_button.pack(side=tk.LEFT, padx=(10, 0))

# Weather display frame
weather_frame = ttk.Frame(main_frame)
weather_frame.pack(fill=tk.X, pady=(0, 20))

weather_icon = ttk.Label(weather_frame)
weather_icon.pack(side=tk.LEFT)

temp_label = ttk.Label(weather_frame, font=("Helvetica", 36))
temp_label.pack(side=tk.LEFT, padx=(20, 0))

info_frame = ttk.Frame(weather_frame)
info_frame.pack(side=tk.LEFT, padx=(20, 0))

humidity_label = ttk.Label(info_frame, font=("Helvetica", 12))
humidity_label.pack(anchor=tk.W)

desc_label = ttk.Label(info_frame, font=("Helvetica", 12))
desc_label.pack(anchor=tk.W)

# Plot frame
plot_frame = ttk.Frame(main_frame)
plot_frame.pack(fill=tk.BOTH, expand=True)

root.mainloop() 